In [3]:
#with improvment

import numpy as np
import random
import pickle
import joblib
import pandas as pd
import time


random.seed(42)
np.random.seed(42)

# Load the saved XGBoost models for calculating validation time in each peer
with open('VP1_xgboost_model.pkl', 'rb') as model_file:
    VP1_xgboost_model = pickle.load(model_file)
with open('VP0_xgboost_model.pkl', 'rb') as model_file:
    VP0_xgboost_model = pickle.load(model_file)

# Load the decision tree model for latency calculation
LA_tree_model = joblib.load('LA_tree_model.pkl')

# Parameters
n = 5  # Number of transactions
m = 2  # Number of committing nodes
lb = 2  # Minimum transactions per block
ub = 200  # Maximum transactions per block
cb = 10240300000  # Maximum block size in bytes
BW = [5000, 5000]  # Bandwidth for each committing node
s = 153600  # Size of each transaction in bytes

# Derived Parameters
nb = (n + lb - 1) // lb  # Estimated number of blocks

# Helper functions for Genetic Algorithm
def create_individual():
    """Create an individual where each transaction is randomly assigned to a block."""
    return [random.randint(1, nb) for _ in range(n)]

def calculate_fitness(individual):
    """Calculate the fitness of an individual using XGBoost for validation, decision tree for latency, and quadratic functions for committing time."""
    block_sizes = [0] * nb
    storing_time, latency_time = 0, 0

    # Count the number of transactions in each block
    for i in range(n):
        block = individual[i] - 1
        block_sizes[block] += 1

    # Iterate over each block to calculate storing and latency times if constraints are met
    for j in range(nb):
        block_size_in_bytes = block_sizes[j] * s  # Calculate the block size in bytes

        # Ensure the block meets both the transaction count and byte size constraints
        if block_sizes[j] <= ub and block_size_in_bytes <= cb:
            data = {
                'Number_of_trx_in_block': [block_sizes[j]], 
                'Size_of_trx_in_block': [s],
                'Size_of_block': [block_size_in_bytes],
                'Bandwidth': [BW[0]]
            }
            df = pd.DataFrame(data)

            # Calculate validation time using XGBoost models
            validation_time_k0 = VP0_xgboost_model.predict(df)[0]
            validation_time_k1 = VP1_xgboost_model.predict(df)[0]

            # Calculate committing time for P0 and P1
            committing_time_k0 = (
                (-1.13290869074807e-07 * block_sizes[j] ** 2) +
                (1.5700675227814848e-09 * block_sizes[j] * BW[0]) +
                (1.1592011649579651e-10 * BW[0] ** 2) +
                (1.2823599531905055e-05 * block_sizes[j]) +
                (-3.045685562355441e-07 * BW[0]) +
                0.0017000585611899274
            )
            
            committing_time_k1 = (
                (-1.1295068444618937e-07 * block_sizes[j] ** 2) +
                (1.0545345201597023e-09 * block_sizes[j] * BW[1]) +
                (9.551926125427123e-11 * BW[1] ** 2) +
                (1.318708108144203e-05 * block_sizes[j]) +
                (-2.5780158023646625e-07 * BW[1]) +
                0.001665642162405443
            )

            # Storing time is the max of validation and committing times across nodes
            max_storing_time = max(
                (validation_time_k0 + committing_time_k0),
                (validation_time_k1 + committing_time_k1)
            )
            storing_time += max_storing_time

            # Latency time calculation using the decision tree model
            latency_time += LA_tree_model.predict(df)[0]

    return storing_time + latency_time

def mutate(individual):
    """Mutate an individual by randomly changing a transaction's block assignment."""
    i = random.randint(0, n - 1)
    individual[i] = random.randint(1, nb)
    return individual

# Genetic Algorithm parameters
population_size = 50
generations = 100
mutation_rate = 0.05
# time_limit = 100  # Set a time limit of 5 minutes (300 seconds) - Commented as per request

# Define a threshold for generations without improvement
stagnation_threshold = 10
generations_since_improvement = 0

# Initialize population
population = [create_individual() for _ in range(population_size)]
best_solution, best_fitness = None, float('inf')

    

# Run the Genetic Algorithm
for generation in range(generations):
    # Calculate fitness for each individual
    fitness_scores = [calculate_fitness(ind) for ind in population]
    
    # Selection with normalized probabilities
    total_fitness = sum(fitness_scores)
    inverted_fitness = [1 / f if f > 0 else 1 for f in fitness_scores]
    selection_probs = [f / sum(inverted_fitness) for f in inverted_fitness]
    selected_indices = np.random.choice(population_size, size=(population_size // 2) * 2, p=selection_probs, replace=True)

    # Crossover
    new_population = []
    for i in range(0, len(selected_indices), 2):
        parent1, parent2 = population[selected_indices[i]], population[selected_indices[i + 1]]
        crossover_point = random.randint(1, n - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        new_population.extend([child1, child2])
    
    # Mutation
    for individual in new_population:
        if random.random() < mutation_rate:
            mutate(individual)

    # Update population
    population = new_population

    # Track the best solution
    min_fitness = min(fitness_scores)
    if min_fitness < best_fitness:
        best_fitness = min_fitness
        best_solution = population[fitness_scores.index(min_fitness)]
        generations_since_improvement = 0  # Reset the stagnation counter
    else:
        generations_since_improvement += 1
    
    # Stop if no improvement in the last few generations
    if generations_since_improvement >= stagnation_threshold:
        print("No improvement, stopping optimization.")
        break

#    print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

# Display the best solution found
print("Best solution found:")
print("Fitness of best solution:", best_fitness)

# Calculate and display the size of each block in the best solution
block_sizes = [0] * nb
for transaction in best_solution:
    block = transaction - 1  # Adjust for zero-based indexing
    block_sizes[block] += 1

#print("\nBlock sizes for the best solution:")
#for j, size in enumerate(block_sizes, start=1):
#    if size > 0:  # Only print blocks that have transactions assigned
#        print(f"Block {j}: {size} transactions")

# Find and display the maximum block size
max_block_size = max(best_solution)
print(f"\nThe Best size of block is: {max_block_size}")


No improvement, stopping optimization.
Best solution found:
Fitness of best solution: 1.6694547077456892

The Best size of block is: 3
